In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
reg_file = "regression.csv"
reg_data = pd.read_csv(reg_file)

In [4]:
reg_data.describe()

,read_num_transformed,per_cost,travel_days,photo_num,person_num_三五好友,person_num_亲子,person_num_家庭,person_num_情侣,person_num_独自一人,person_num_闺蜜,is_weekend,title_length,playmode1,playmode2
count,1263.000000,1.263000e+03,1263.000000,1263.000000,1263.000000,1263.000000,1263.000000,1263.000000,1263.000000,1263.000000,1263.000000,1263.000000,1263.000000,1263.000000
mean,8341.736342,8.172789e+04,4.531275,88.590657,0.261283,0.079968,0.155186,0.198733,0.171021,0.133808,0.329375,12.261283,0.550277,0.821853
std,18077.039828,2.813762e+06,2.247822,94.683999,0.439508,0.271352,0.362225,0.399205,0.376677,0.340581,0.470172,5.124579,0.497663,0.382788
min,231.000000,1.200000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,2375.500000,1.100000e+03,3.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,1.000000
50%,4143.000000,2.000000e+03,4.000000,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,1.000000
75%,7778.000000,3.500000e+03,6.000000,123.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,16.000000,1.000000,1.000000
max,461000.000000,1.000000e+08,9.000000,1068.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,36.000000,1.000000,1.000000


array([1.80760404, 2.19885551, 2.28219125, 1.9809912 , 2.17788485,
       2.12790819, 2.08568022, 2.15164097, 2.10038768, 2.1079532 ,
       2.37950683, 2.21120871, 2.16896876, 2.1447989 , 2.05821972,
       2.13818793, 2.32629769, 2.07460112, 2.25620734, 2.14888288,
       2.08282956, 2.03924226, 2.1772517 , 2.1771681 , 2.16490182,
       2.10443685, 2.29288697, 2.23992874, 2.16326725, 2.0488226 ,
       2.12369069, 1.99616046, 2.23992874, 1.92018162, 2.08229956,
       2.00312194, 2.05435453, 1.88663709, 2.11423437, 2.00838069,
       2.36053647, 2.0451883 , 2.17039072, 1.93206434, 2.08876796,
       2.10038768, 2.24841447, 1.99572784, 2.17992392, 2.22032681,
       2.3884636 , 1.96621932, 1.78349921, 2.16127484, 2.19663205,
       2.07473243, 2.15549103, 2.08056974, 2.0236372 , 2.17464471,
       1.97288995, 2.36286399, 2.01881622, 2.19393384, 2.13222304,
       2.02996484, 2.15787979, 2.09505979, 2.03541006, 1.79646148,
       2.17915585, 2.03918242, 2.10097543, 2.05451169, 1.73817

In [26]:
# 划分训练集和测试集
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

num_data=['per_cost', 'travel_days', 'photo_num','title_length']
#转换流水线，可以在其中添加一些其他自定义的属性
num_pipeline=Pipeline([  # 数值变量标准化
  ("std_scaler",StandardScaler())
])
reg_data.loc[:,num_data]=num_pipeline.fit_transform(reg_data.loc[:,num_data])

reg_data.loc[:,"read_num_transformed"]=np.log(reg_data.loc[:,"read_num_transformed"])  # 对响应变量进行对数变换

train,test=train_test_split(reg_data,test_size=0.3,random_state=42)
train,test=train.values,test.values
X_train,Y_train=train[:,1:],train[:,0]
X_test,Y_test=test[:,1:],test[:,0]
X_train.shape,Y_train.shape

((884, 13), (884,))

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

lin_reg=LinearRegression()
lin_scores=cross_val_score(lin_reg,X_train,Y_train,scoring="neg_mean_squared_error",cv=10)
def display_scores(scores):
    print("Scores:",scores)
    print("Mean:",scores.mean())
    print("Std:",scores.std())

display_scores(-lin_scores)

lin_reg.fit(X_train,Y_train)
#打印系数
print("线性回归系数：",lin_reg.coef_)
lin_predictions=lin_reg.predict(X_test)
lin_mse=mean_squared_error(Y_test,lin_predictions)
print("在测试集上的MSE：",lin_mse)

Scores: [0.00246487 0.00328758 0.00293046 0.0024186  0.00243027 0.00180722
 0.00278517 0.00238917 0.00218498 0.00177169]
Mean: 0.0024470017453824587
Std: 0.0004464792043783301
线性回归系数： [-9.38190350e+00  2.46157906e-03  2.03942088e-02 -8.57601666e-04
  1.28230519e-02 -1.98846951e-03  1.08786504e-04 -6.18392597e-03
 -3.90184125e-03  1.59748512e-03  8.78464684e-03  1.57833376e-02
  1.08903907e-02]
在测试集上的MSE： 293.47945259315145


In [ ]:
# 线性回归模型